# P05: Congestion Phase Transition & Universality (PhD Level)

## 1. Introduction: Traffic as a Many-Body System

Traffic flow exhibits a non-equilibrium phase transition from **Free Flow** (gas-like) to **Congested Flow** (liquid/solid-like). 
The **Nagel-Schreckenberg (NaSch)** model is the Ising model of traffic: simple, yet capturing the essential physics of spontaneous symmetry breaking (jam formation).

In this PhD-level notebook, we will not just "see" the jam. We will prove the nature of this phase transition using **Finite Size Scaling (FSS)**, a rigorous technique to extract universal critical exponents from finite simulations.

### Core Mappings
| Traffic Physics | Statistical Physics |
| :--- | :--- |
| Car Density $\rho$ | Temperature $T$ (Control Parameter) |
| Average Flow $J$ | Order Parameter $M$ (Magnetization) |
| Variance of Flow $\chi$ | Susceptibility $\chi$ (Response) |
| Critical Density $\rho_c$ | Critical Temperature $T_c$ |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## 2. The Model: Nagel-Schreckenberg (NaSch)

Rules for update (parallel):
1. **Accelerate**: $v_i \to \min(v_i + 1, v_{\max})$
2. **Decelerate**: $v_i \to \min(v_i, d_i - 1)$ (avoid collision, $d_i$ is gap)
3. **Randomization**: With prob $p$, $v_i \to \max(v_i - 1, 0)$ (human error/delay)
4. **Move**: $x_i \to x_i + v_i$

In [ ]:
def simulate_nasch(L, rho, v_max=5, p=0.3, steps=1000, trans=200):
    N = int(L * rho)
    # Initialize uniformly
    pos = np.sort(np.random.choice(L, N, replace=False))
    vel = np.random.randint(0, v_max + 1, N)
    
    params = {'L': L, 'N': N, 'v_max': v_max, 'p': p}
    
    mean_vels = []
    flows = []
    
    for t in range(steps):
        # Calculate gaps (periodic boundary)
        gaps = (np.roll(pos, -1) - pos) % L
        # The last car's gap needs correction for PBC
        gaps[-1] = (pos[0] + L) - pos[-1]
        # Distance is gap - 1 (empty cells)
        dist = gaps - 1
        
        # 1. Accelerate
        vel = np.minimum(vel + 1, v_max)
        # 2. Decelerate
        vel = np.minimum(vel, dist)
        # 3. Randomize
        mask = np.random.rand(N) < p
        vel = np.maximum(vel - mask, 0)
        # 4. Move
        pos = (pos + vel) % L
        
        if t > trans:
            # Record observables
            # Flow J = density * mean_velocity = (N/L) * avg(v)
            # Or simply: sum(v) / L (flux through a point)
            J = np.sum(vel) / L
            flows.append(J)
            
    return np.array(flows)

## 3. Finite Size Scaling (FSS)

Near a critical point, the correlation length diverges: $\xi \sim |\rho - \rho_c|^{-\nu}$.
In a finite system of size $L$, scaling cuts off when $\xi \sim L$.

We define the "susceptibility" $\chi$ as the fluctuation of the order parameter (Flow $J$):
$$ \chi = L \cdot \text{Var}(J) $$

According to FSS hypothesis, near $\rho_c$:
$$ \chi(L, \rho) \sim L^{\gamma/\nu} \tilde{\chi}\left((\rho - \rho_c)L^{1/\nu}\right) $$

If we find the correct exponents $\gamma, \nu$ and critical point $\rho_c$, plotting $\chi L^{-\gamma/\nu}$ vs $(\rho - \rho_c)L^{1/\nu}$ should make curves for different $L$ **collapse onto a single master curve**.

In [ ]:
# Simulation Parameters
Ls = [100, 200, 400] # Different system sizes
rhos = np.linspace(0.05, 0.5, 30)
v_max = 5
p = 0.3 # Moderate noise

results = {}

print("Running simulations...")
for L in Ls:
    chis = []
    Js = []
    for rho in tqdm(rhos, desc=f"L={L}"):
        flows = simulate_nasch(L, rho, v_max, p, steps=2000, trans=500)
        # Susceptibility = L * Var(J)
        chi = L * np.var(flows)
        chis.append(chi)
        Js.append(np.mean(flows))
    results[L] = {'rho': rhos, 'J': Js, 'chi': chis}

# First plot: Raw data
plt.figure(figsize=(10, 5))
for L in Ls:
    plt.plot(results[L]['rho'], results[L]['chi'], 'o-', label=f'L={L}')
plt.xlabel(r'Density $\rho$')
plt.ylabel(r'Susceptibility $\chi$')
plt.title('Fluctuations peak near Critical Point')
plt.legend()
plt.grid(True)
plt.show()

## 4. Data Collapse (The "Gold Standard" of Proof)

Now we try to collapse these curves. This requires tuning $\rho_c, \nu, \gamma$ manually or via optimization.
For 1D traffic models like NaSch, values are often non-trivial (unlike 2D Ising).
Empirically, for $v_{max}=5, p=0.3$, the transition is smoothened, but we can look for the best overlap.

In [ ]:
def plot_fss(results, Ls, rho_c, nu, gamma):
    plt.figure(figsize=(8, 6))
    for L in Ls:
        rho = results[L]['rho']
        chi = np.array(results[L]['chi'])
        
        # X axis: (rho - rho_c) * L^(1/nu)
        x_scaled = (rho - rho_c) * (L**(1/nu))
        # Y axis: chi * L^(-gamma/nu)
        y_scaled = chi * (L**(-gamma/nu))
        
        plt.plot(x_scaled, y_scaled, 'o-', label=f'L={L}', alpha=0.7)
        
    plt.xlabel(r'$(\rho - \rho_c) L^{1/\nu}$')
    plt.ylabel(r'$\chi L^{-\gamma/\nu}$')
    plt.title(f'Data Collapse Check\n$\rho_c={rho_c}, \nu={nu}, \gamma={gamma}$')
    plt.legend()
    plt.grid(True)
    plt.show()

# Try some guess values (you can interactively tune these)
# Critical density is usually around 1/(v_max + 1) to 1/v_max depending on noise
# For v_max=5, maybe around 0.1-0.2
plot_fss(results, Ls, rho_c=0.15, nu=1.5, gamma=1.0)